In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import ROOT
from ROOT import TFile, TTree, TH1F, TCanvas, TAxis, TLegend, TTreeReader, TTreeReaderValue
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
import multiprocessing
import time
import concurrent.futures

Welcome to JupyROOT 6.22/06


In [2]:
from joblib import dump, load
clf = load('BDT_model_QCD_sig0p3_matching3.joblib') 

In [3]:
def BDT_sort(event_record, evtnum, index, ismatched_arr, variable2use):
    global counter
    if not sum(evtnum) / 3 == evtnum[0]:
        print("Check if 3 candidates are coming from the same event")
        print(evtnum)
    best_score = 0
    best_index = -99
    df_temp = pd.DataFrame.from_records(event_record, columns=variable2use)
    score_BDT_temp = clf.predict(df_temp)
#     print(score_BDT_temp)
#     print(df_temp)
#     print(index)
#     print(ismatched_arr)
    for i, score in enumerate(score_BDT_temp.ravel()):
        if score > best_score:
            best_score = score
            best_index = index[i]
#     print(best_score, best_index)
    return best_score, best_index

def select_trijet(sample, batch_size, ibatch, isSig):
    inFile = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
    inTree = inFile.Get("Events")
    variable2use = ['dijet_eta','dijet_phi','dR_jj','dEta_jj','dPhi_jj','dR_j0j2','dEta_j0j2','dPhi_j0j2','dR_j1j2','dEta_j1j2','dPhi_j1j2',
                    'jet_eta_0','jet_phi_0','jet_ptoverm_0','jet_eta_1','jet_phi_1','jet_ptoverm_1','jet_eta_2',
                    'jet_phi_2','jet_ptoverm_2','dR_jj_j','dEta_jj_j','dPhi_jj_j','jet_ptoverM_0','jet_ptoverM_1','jet_ptoverM_2',
                    'dijet_ptoverM']
    outFile = TFile(f"{sample}_BDT_Ranking_{ibatch}.root","RECREATE")
    outTree = TTree("Events","Events")
    
    dijet_eta = np.empty((1), dtype="float32")
    dijet_phi = np.empty((1), dtype="float32")
    dR_jj = np.empty((1), dtype="float32")
    dEta_jj = np.empty((1), dtype="float32")
    dPhi_jj = np.empty((1), dtype="float32")
    jet_eta_0 = np.empty((1), dtype="float32")
    jet_phi_0 = np.empty((1), dtype="float32")
    jet_ptoverm_0 = np.empty((1), dtype="float32")
    jet_eta_1 = np.empty((1), dtype="float32")
    jet_phi_1 = np.empty((1), dtype="float32")
    jet_ptoverm_1 = np.empty((1), dtype="float32")
    jet_eta_2 = np.empty((1), dtype="float32")
    jet_phi_2 = np.empty((1), dtype="float32")
    jet_ptoverm_2 = np.empty((1), dtype="float32")
    dR_jj_j = np.empty((1), dtype="float32")
    dEta_jj_j = np.empty((1), dtype="float32")
    dPhi_jj_j = np.empty((1), dtype="float32")
    jet_ptoverM_0 = np.empty((1), dtype="float32")
    jet_ptoverM_1 = np.empty((1), dtype="float32")
    jet_ptoverM_2 = np.empty((1), dtype="float32")
    dijet_ptoverM = np.empty((1), dtype="float32")
    M_jjj = np.empty((1), dtype="float32")
    m_jj = np.empty((1), dtype="float32")
    score_BDT = np.empty((1), dtype="float32")
    isMatched = np.empty((1), dtype="int32")

    outTree.Branch("dijet_eta", dijet_eta, "dijet_eta/F")
    outTree.Branch("dijet_phi", dijet_phi, "dijet_phi/F")
    outTree.Branch("dR_jj", dR_jj, "dR_jj/F")
    outTree.Branch("dEta_jj", dEta_jj, "dEta_jj/F")
    outTree.Branch("dPhi_jj", dPhi_jj, "dPhi_jj/F")
    outTree.Branch("jet_eta_0", jet_eta_0, "jet_eta_0/F")
    outTree.Branch("jet_phi_0", jet_phi_0, "jet_phi_0/F")
    outTree.Branch("jet_ptoverm_0", jet_ptoverm_0, "jet_ptoverm_0/F")
    outTree.Branch("jet_eta_1", jet_eta_1, "jet_eta_1/F")
    outTree.Branch("jet_phi_1", jet_phi_1, "jet_phi_1/F")
    outTree.Branch("jet_ptoverm_1", jet_ptoverm_1, "jet_ptoverm_1/F")
    outTree.Branch("jet_eta_2", jet_eta_2, "jet_eta_2/F")
    outTree.Branch("jet_phi_2", jet_phi_2, "jet_phi_2/F")
    outTree.Branch("jet_ptoverm_2", jet_ptoverm_2, "jet_ptoverm_2/F")
    outTree.Branch("dR_jj_j", dR_jj_j, "dR_jj_j/F")
    outTree.Branch("dEta_jj_j", dEta_jj_j, "dEta_jj_j/F")
    outTree.Branch("dPhi_jj_j", dPhi_jj_j, "dPhi_jj_j/F")
    outTree.Branch("jet_ptoverM_0", jet_ptoverM_0, "jet_ptoverM_0/F")
    outTree.Branch("jet_ptoverM_1", jet_ptoverM_1, "jet_ptoverM_1/F")
    outTree.Branch("jet_ptoverM_2", jet_ptoverM_2, "jet_ptoverM_2/F")
    outTree.Branch("dijet_ptoverM", dijet_ptoverM, "dijet_ptoverM/F")
    outTree.Branch("M_jjj", M_jjj, "M_jjj/F")
    outTree.Branch("m_jj", m_jj, "m_jj/F")
    outTree.Branch("score_BDT", score_BDT, "score_BDT/F")
    outTree.Branch("isMatched", isMatched, "isMatched/I")

    t_start = time.time()
    evtnum = []
    index = []
    event_record = []
    ismatched_arr = []
    evt_start = ibatch*batch_size
    evt_end = (ibatch+1)*batch_size
    print(evt_start, evt_end)
    if((ibatch+1)*batch_size > inTree.GetEntries()):
        evt_end = inTree.GetEntries()
    for ievt in range(evt_start, evt_end):
        inTree.GetEntry(ievt)
#         if ievt%1000 == 0:
#             print("Processing: ",ievt)
#             # time check:
#             t_end = time.time()
#             speed = (t_end - t_start)/(ievt+1)*1000
#             print(f"Avg. speed: {speed}s/1k candidates".format(speed))
#         if ievt > 100000: break

        if (ievt - evt_start)%30000 == 0:
            # time check:
            t_end = time.time()
            speed = (t_end - t_start)/(ievt - evt_start+1)*1000
            t_remain = (evt_end - ievt) / 1000 * speed / 60
            print(f"Batch #{ibatch} >> Avg. speed: {speed}s/1k candidates, time remaining: {t_remain}mins\n".format(ibatch, speed, t_remain))

        # make event record for BDT classification
        trijet_record = []
        for var in variable2use:
            trijet_record.append(getattr(inTree, var))
        if len(event_record) < 3:
            event_record.append(trijet_record)
            evtnum.append(inTree.evt_num)
            index.append(ievt)
            if isSig == 1:
                ismatched_arr.append(inTree.gen_dijet_matched)
            else:
                ismatched_arr.append(-99)
            if(len(event_record) == 3):
                best_score, best_index = BDT_sort(event_record, evtnum, index, ismatched_arr, variable2use)
                inTree.GetEntry(best_index)
                dijet_eta[0] = inTree.dijet_eta
                dijet_phi[0] = inTree.dijet_phi
                dR_jj[0] = inTree.dR_jj
                dEta_jj[0] = inTree.dEta_jj
                dPhi_jj[0] = inTree.dPhi_jj
                jet_eta_0[0] = inTree.jet_eta_0
                jet_phi_0[0] = inTree.jet_phi_0
                jet_ptoverm_0[0] = inTree.jet_ptoverm_0
                jet_eta_1[0] = inTree.jet_eta_1
                jet_phi_1[0] = inTree.jet_phi_1
                jet_ptoverm_1[0] = inTree.jet_ptoverm_1
                jet_eta_2[0] = inTree.jet_eta_2
                jet_phi_2[0] = inTree.jet_phi_2
                jet_ptoverm_2[0] = inTree.jet_ptoverm_2
                dR_jj_j[0] = inTree.dR_jj_j
                dEta_jj_j[0] = inTree.dEta_jj_j
                dPhi_jj_j[0] = inTree.dPhi_jj_j
                jet_ptoverM_0[0] = inTree.jet_ptoverM_0
                jet_ptoverM_1[0] = inTree.jet_ptoverM_1
                jet_ptoverM_2[0] = inTree.jet_ptoverM_2
                dijet_ptoverM[0] = inTree.dijet_ptoverM
                M_jjj[0] = inTree.M_jjj
                m_jj[0] = inTree.m_jj
                score_BDT[0] = best_score
                if isSig == 1:
                    isMatched[0] = inTree.gen_dijet_matched
#                     print(isMatched[0])
                else:
                    isMatched[0] = -99
                outTree.Fill()
                event_record.clear()
                evtnum.clear()
                index.clear()
                ismatched_arr.clear()
#     print("Number of trijet candidates selected: ", outTree.GetEntries())
    outFile.cd()
    outTree.Write()
    outFile.Write()
    outFile.Close()
    print(f"Finished processing of batch {ibatch}/n")
    return 0

In [4]:
# QCD processing
if __name__ == '__main__':
    
    for sample in ["QCD_Pt_600to800"]:
#     for sample in ["QCD_Pt_300to470","QCD_Pt_470to600","QCD_Pt_600to800"]:
#     for sample in ["QCD_Pt_800to1000","QCD_Pt_1000to1400","QCD_Pt_1400to1800",
#                    "QCD_Pt_1800to2400","QCD_Pt_2400to3200","QCD_Pt_3200toInf"]:
        isSig = 0
        temp_file = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
        temp_tree = temp_file.Get("Events")
        tot_evts = temp_tree.GetEntries()
        print(tot_evts)
        expect_time = 0.3 # in hrs
        known_speed = 4 # sec per 1k candidates
        evt_batch = int(expect_time * 3600 / known_speed * 1000)
        if(evt_batch%3 != 0):
            print("!!!!")
        num_batch = math.ceil(tot_evts / evt_batch)
        print(f"Number of Candidates to be processed: {tot_evts}")
        print(f"Candidates to be processed per batch: {evt_batch}")
        print(f"Number of batches to be processed: {num_batch}")

        main_start = time.time()

    #     multiprocessing way
    #     p1 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_1))
    #     p2 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_2))
    #     p1.start()
    #     p1.join()
    #     p2.start()
    #     p2.join()
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = [executor.submit(select_trijet, sample, evt_batch, ibatch, isSig) for ibatch in range(num_batch)]
            status = [r.result() for r in results]
            print(status)

        print(f"Time used: {round(time.time() - main_start, 2)}")

17687280
Number of Candidates to be processed: 17687280
Candidates to be processed per batch: 270000
Number of batches to be processed: 66
0270000 1890000270000 
 5400005400002430000 2160000
 8100003780000

 2700000
108000040500004050000
 4320000810000  13500001080000

1350000
 Batch #0 >> Avg. speed: 49.453020095825195s/1k candidates, time remaining: 222.53859043121338mins
297000016200002160000

 Batch #1 >> Avg. speed: 45.18413543701172s/1k candidates, time remaining: 203.32860946655273mins
 3240000
2430000
324000027000001620000 
2970000
 1890000 35100003510000
 
3780000
Batch #9 >> Avg. speed: 392.93718338012695s/1k candidates, time remaining: 1768.2173252105713mins

Batch #10 >> Avg. speed: 394.181489944458s/1k candidates, time remaining: 1773.816704750061mins
Batch #14 >> Avg. speed: 433.7790012359619s/1k candidates, time remaining: 1952.0055055618286mins

Batch #5 >> Avg. speed: 424.8030185699463s/1k candidates, time remaining: 1911.6135835647583mins

Batch #7 >> Avg. speed: 453.


Batch #12 >> Avg. speed: 4.008465230112475s/1k candidates, time remaining: 8.01693046022495mins

Batch #13 >> Avg. speed: 4.048040630478845s/1k candidates, time remaining: 8.09608126095769mins

Batch #8 >> Avg. speed: 4.064808347953298s/1k candidates, time remaining: 8.129616695906597mins

Batch #9 >> Avg. speed: 4.0712130039453704s/1k candidates, time remaining: 8.142426007890741mins

Batch #2 >> Avg. speed: 4.071373603219049s/1k candidates, time remaining: 8.142747206438099mins

Batch #1 >> Avg. speed: 4.073307912045052s/1k candidates, time remaining: 8.146615824090103mins

Batch #10 >> Avg. speed: 4.095344756252142s/1k candidates, time remaining: 8.190689512504283mins

Batch #5 >> Avg. speed: 4.115809283327516s/1k candidates, time remaining: 8.231618566655031mins

Batch #7 >> Avg. speed: 4.1163562770690945s/1k candidates, time remaining: 8.232712554138189mins

Batch #6 >> Avg. speed: 4.130667015277251s/1k candidates, time remaining: 8.261334030554503mins

Batch #3 >> Avg. speed: 4.

Finished processing of batch 5/n
7830000 8100000
Batch #29 >> Avg. speed: 384.4618797302246s/1k candidates, time remaining: 1730.0784587860107mins

Finished processing of batch 11/n
8100000 8370000
Batch #30 >> Avg. speed: 390.46764373779297s/1k candidates, time remaining: 1757.1043968200684mins

Finished processing of batch 0/n
8370000 8640000
Batch #31 >> Avg. speed: 453.2759189605713s/1k candidates, time remaining: 2039.7416353225708mins

Batch #16 >> Avg. speed: 4.36386853542921s/1k candidates, time remaining: 17.45547414171684mins

Batch #17 >> Avg. speed: 4.258579049307976s/1k candidates, time remaining: 17.034316197231902mins

Batch #28 >> Avg. speed: 3.8414068299290944s/1k candidates, time remaining: 15.365627319716378mins

Batch #20 >> Avg. speed: 4.058522139583872s/1k candidates, time remaining: 16.23408855833549mins

Batch #23 >> Avg. speed: 4.092511596379608s/1k candidates, time remaining: 16.37004638551843mins

Batch #18 >> Avg. speed: 4.379960116670631s/1k candidates, tim


Batch #29 >> Avg. speed: 4.36683942341477s/1k candidates, time remaining: 8.73367884682954mins

Batch #16 >> Avg. speed: 4.234440533941495s/1k candidates, time remaining: 6.351660800912242mins

Batch #17 >> Avg. speed: 4.187350118134618s/1k candidates, time remaining: 6.2810251772019265mins

Batch #28 >> Avg. speed: 4.159177393576332s/1k candidates, time remaining: 6.238766090364498mins

Batch #25 >> Avg. speed: 4.198222623107247s/1k candidates, time remaining: 6.2973339346608705mins

Batch #20 >> Avg. speed: 4.230817905616145s/1k candidates, time remaining: 6.346226858424217mins

Batch #21 >> Avg. speed: 4.232938107875668s/1k candidates, time remaining: 6.349407161813502mins

Batch #19 >> Avg. speed: 4.265377892002061s/1k candidates, time remaining: 6.398066838003091mins

Batch #24 >> Avg. speed: 4.249770022991754s/1k candidates, time remaining: 6.374655034487631mins

Batch #27 >> Avg. speed: 4.2418225983864835s/1k candidates, time remaining: 6.362733897579726mins

Batch #23 >> Avg. 


Batch #42 >> Avg. speed: 4.529193800165869s/1k candidates, time remaining: 18.116775200663476mins

Batch #41 >> Avg. speed: 4.566612841586209s/1k candidates, time remaining: 18.266451366344835mins

Batch #44 >> Avg. speed: 4.4776826162170735s/1k candidates, time remaining: 17.910730464868298mins

Batch #45 >> Avg. speed: 4.508707691107562s/1k candidates, time remaining: 18.034830764430247mins

Batch #46 >> Avg. speed: 4.390478825546106s/1k candidates, time remaining: 17.561915302184424mins

Batch #47 >> Avg. speed: 4.422113190594961s/1k candidates, time remaining: 17.688452762379843mins

Batch #36 >> Avg. speed: 4.007786818836985s/1k candidates, time remaining: 14.027253865929449mins

Batch #34 >> Avg. speed: 4.384336494445419s/1k candidates, time remaining: 15.345177730558966mins

Batch #33 >> Avg. speed: 4.432000254963249s/1k candidates, time remaining: 15.512000892371372mins

Batch #32 >> Avg. speed: 4.465355535115471s/1k candidates, time remaining: 15.628744372904148mins

Batch #3


Batch #45 >> Avg. speed: 4.384788413662934s/1k candidates, time remaining: 6.5771826204944mins

Batch #47 >> Avg. speed: 4.387016389793301s/1k candidates, time remaining: 6.580524584689951mins

Batch #36 >> Avg. speed: 4.151743446447572s/1k candidates, time remaining: 4.151743446447572mins

Batch #33 >> Avg. speed: 4.285393803182817s/1k candidates, time remaining: 4.285393803182817mins

Batch #32 >> Avg. speed: 4.3292715273801665s/1k candidates, time remaining: 4.3292715273801665mins

Batch #34 >> Avg. speed: 4.324842520322933s/1k candidates, time remaining: 4.324842520322933mins

Batch #39 >> Avg. speed: 4.205363985952891s/1k candidates, time remaining: 4.205363985952891mins

Batch #43 >> Avg. speed: 4.209458382075026s/1k candidates, time remaining: 4.209458382075026mins

Batch #37 >> Avg. speed: 4.264120246096333s/1k candidates, time remaining: 4.264120246096333mins

Batch #44 >> Avg. speed: 4.242393132747371s/1k candidates, time remaining: 4.242393132747371mins

Batch #35 >> Avg. s


Batch #56 >> Avg. speed: 4.316703504058624s/1k candidates, time remaining: 15.108462264205182mins

Batch #58 >> Avg. speed: 4.196531921503446s/1k candidates, time remaining: 14.687861725262062mins

Batch #59 >> Avg. speed: 4.169958205730191s/1k candidates, time remaining: 14.59485372005567mins

Batch #61 >> Avg. speed: 4.134538878945676s/1k candidates, time remaining: 14.470886076309869mins

Batch #60 >> Avg. speed: 4.261201438211453s/1k candidates, time remaining: 14.914205033740084mins

Batch #62 >> Avg. speed: 4.147023936593193s/1k candidates, time remaining: 14.514583778076176mins

Batch #63 >> Avg. speed: 4.12914083452257s/1k candidates, time remaining: 14.451992920828994mins

Batch #48 >> Avg. speed: 4.223499905685456s/1k candidates, time remaining: 12.670499717056368mins

Batch #49 >> Avg. speed: 4.305190859160843s/1k candidates, time remaining: 12.91557257748253mins

Batch #50 >> Avg. speed: 4.381752227674867s/1k candidates, time remaining: 13.1452566830246mins

Batch #52 >> A


Batch #60 >> Avg. speed: 4.373595959582752s/1k candidates, time remaining: 4.373595959582752mins

Batch #59 >> Avg. speed: 4.388513125966664s/1k candidates, time remaining: 4.388513125966664mins

Batch #48 >> Avg. speed: 4.18336393306645s/1k candidates, time remaining: 2.091681966533225mins

Batch #63 >> Avg. speed: 4.339277631612588s/1k candidates, time remaining: 4.339277631612588mins

Batch #49 >> Avg. speed: 4.292210079397853s/1k candidates, time remaining: 2.1461050396989263mins

Batch #55 >> Avg. speed: 4.206407592741458s/1k candidates, time remaining: 2.103203796370729mins

Batch #56 >> Avg. speed: 4.208511308058288s/1k candidates, time remaining: 2.104255654029144mins

Batch #50 >> Avg. speed: 4.355385600714637s/1k candidates, time remaining: 2.1776928003573186mins

Batch #54 >> Avg. speed: 4.27401943451483s/1k candidates, time remaining: 2.137009717257415mins

Batch #57 >> Avg. speed: 4.222217248615926s/1k candidates, time remaining: 2.111108624307963mins

Batch #51 >> Avg. s

In [ ]:
# Signal processing
if __name__ == '__main__':
    
    for sample in ["Res1ToRes2GluTo3Glu_M1-3000_R-0p7","Res1ToRes2GluTo3Glu_M1-5000_R-0p7","Res1ToRes2GluTo3Glu_M1-7000_R-0p7"]:
        isSig = 1
        temp_file = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
        temp_tree = temp_file.Get("Events")
        tot_evts = temp_tree.GetEntries()

        expect_time = 8 # in mins
        known_speed = 2.4 # sec per 1k candidates
        evt_batch = int(expect_time * 60 / known_speed * 1000)
        num_batch = math.ceil(tot_evts / evt_batch)
        print(f"Number of Candidates to be processed: {tot_evts}")
        print(f"Candidates to be processed per batch: {evt_batch}")
        print(f"Number of batches to be processed: {num_batch}")

        main_start = time.time()

        batch_size = evt_batch

    #     multiprocessing way
    #     p1 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_1))
    #     p2 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_2))
    #     p1.start()
    #     p1.join()
    #     p2.start()
    #     p2.join()

        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = [executor.submit(select_trijet, sample, batch_size, ibatch, isSig) for ibatch in range(num_batch)]
            status = [r.result() for r in results]
            print(status)

        print(f"Time used: {round(time.time() - main_start, 2)}")

In [ ]:
# Dry-run, speed calculation
sample = "QCD_Pt_300to470"
isSig = 0
temp_file = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
temp_tree = temp_file.Get("Events")
tot_evts = temp_tree.GetEntries()

main_start = time.time()

counter = 0
select_trijet(sample, 50000, 0, isSig)

print(f"Time used: {round(time.time() - main_start, 2)}")